In [ ]:
%pip install semantic-router==0.0.37
%pip install sentence-transformers
%pip install chromadb
%pip install pypdf
%pip install langchain-community

In [2]:
import os
os.environ["COHERE_API_KEY"] = "kUYLefL4SNo36iNh5tF2H1gGi5XfZyTbOBJNknt1"

In [3]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

encoder.encode("Hello, World!")

/Users/anavbo/Desktop/Personal/skin-cancer-detection-app/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


array([-3.81771624e-02,  3.29111032e-02, -5.45941340e-03,  1.43699506e-02,
       -4.02910337e-02, -1.16532490e-01,  3.16876657e-02,  1.91175914e-03,
       -4.26223204e-02,  2.91681141e-02,  4.24266793e-02,  3.20417359e-02,
        2.98447311e-02,  1.09802913e-02, -5.39396070e-02, -5.02772778e-02,
       -2.35078298e-02,  1.07793808e-02, -1.37707949e-01,  4.11499850e-03,
        2.93330923e-02,  6.68411404e-02, -1.53893977e-02,  4.84376550e-02,
       -8.81497487e-02, -1.27268638e-02,  4.14090492e-02,  4.08315063e-02,
       -5.01559079e-02, -5.81250526e-02,  4.88015115e-02,  6.88901171e-02,
        5.87469079e-02,  8.73098895e-03, -1.59182511e-02,  8.51419568e-02,
       -7.81474337e-02, -7.75167868e-02,  2.07238067e-02,  1.61942486e-02,
        3.25106122e-02, -5.34888543e-02, -6.22287765e-02, -2.43146550e-02,
        7.41279405e-03,  2.39777453e-02,  6.36094343e-03,  5.11451177e-02,
        7.27666914e-02,  3.46496850e-02, -5.47711216e-02, -5.93284853e-02,
       -7.16693699e-03,  

In [4]:
import chromadb
import os
client = chromadb.PersistentClient(
    path="/Users/anavbo/Desktop/Personal/skin-cancer-detection-app/RAG/chromadb",
)
collection = client.get_or_create_collection(name="skin-cancer")

In [5]:
import chromadb.utils.embedding_functions as embedding_functions
chroma_encode = embedding_functions.CohereEmbeddingFunction(
    api_key=os.environ["COHERE_API_KEY"], model_name="embed-english-v3.0"
)

In [6]:
# semantic chunking
from semantic_router.splitters import RollingWindowSplitter
from semantic_router.utils.logger import logger
from semantic_router.encoders import CohereEncoder

logger.setLevel("WARNING")
encoder = CohereEncoder()
splitter = RollingWindowSplitter(
    encoder=encoder,
    min_split_tokens=50,
    max_split_tokens=400,
    window_size=2,
    enable_statistics=False,
)

In [7]:
from pypdf import PdfReader
import os
import uuid
path="/Users/anavbo/Desktop/Personal/skin-cancer-detection-app/RAG/documents"
pdf_paths = os.listdir(path)

chunks = []
metadatas = []
ids = []
for doc_path in pdf_paths:
    documents = PdfReader(f"{path}/{doc_path}")
    document = ""
    for doc in documents.pages:
        document += doc.extract_text() + "\n"

    splits = splitter([document])
    for i in range(len(splits)):
        chunk = " ".join(splits[i].docs)
        try:
            pre_chunk = " ".join(splits[i - 1].docs)
        except:
            pre_chunk = ""
        try:
            post_chunk = " ".join(splits[i + 1].docs)
        except:
            post_chunk = ""
        
        metadata = {
            "source": doc_path,
            "pre_chunk": pre_chunk,
            "chunk": chunk,
            "post_chunk": post_chunk
        }
        chunks.append(chunk)
        metadatas.append(metadata)
        ids.append(str(uuid.uuid4().hex))

In [12]:
ids[0]

'a66975655a1a4a9898b93e5904806ace'

In [13]:
collection.add(
    documents = chunks,
    ids = ids,
    metadatas = metadatas,
)

Add of existing embedding ID: a66975655a1a4a9898b93e5904806ace
Insert of existing embedding ID: a66975655a1a4a9898b93e5904806ace


In [ ]:
import chromadb

query = input("What is your query:\n")


client = chromadb.PersistentClient(
    path="/Users/anavbo/Desktop/Personal/skin-cancer-detection-app/RAG/chromadb"
)
collection = client.get_collection("skin-cancer")

print(collection.query(query_texts=query, n_results=5)["data"])